In [58]:
# global var set
import transformers
# model info, change as needed
batch_size = 16
num_epochs = 16

# model_checkpoint = "bert-base-uncased"
# model_checkpoint = "roberta-base"
# model_checkpoint = 'ProsusAI/finbert'
model_checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
metric_name = "f1"
# fileTag = "original-plutchik-v1"
fileTag = 'clean-v1'

# Convert dataset to suitable format
IMPORTANT: please never run this section again if you have your dataset ready!

In [66]:
import pandas as pd
from pandas import DataFrame

trainDatasetOriginal = pd.read_csv(f'../../data/csv_version/dev/emotion/allcharlinepairs-{fileTag}.csv')
testDatasetOriginal = pd.read_csv(f'../../data/csv_version/test/emotion/allcharlinepairs-{fileTag}.csv')

In [67]:
trainDatasetOriginal

,Unnamed: 0,storyid,linenum,char,emotionworkerid,context,sentence,affected,emotion,plutchik
0,0,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann0,NaN,I began making fish curry for my boyfriend and I.,yes,"['joy', 'excited']","{'joy': 3, 'trust': 3, 'fear': 0, 'surprise': ..."
1,1,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann1,NaN,I began making fish curry for my boyfriend and I.,yes,['content'],"{'joy': 2, 'trust': 0, 'fear': 0, 'surprise': ..."
2,2,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,1,I (myself),ann2,NaN,I began making fish curry for my boyfriend and I.,yes,"['hungry', 'anticipation']","{'joy': 2, 'trust': 0, 'fear': 0, 'surprise': ..."
3,3,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,2,I (myself),ann1,I began making fish curry for my boyfriend and I.,I decided not to read a recipe since I've made...,yes,['excited'],"{'joy': 3, 'trust': 3, 'fear': 0, 'surprise': ..."
4,4,a2ddbb50-e45b-4ad3-becf-b2d8475172bf,2,I (myself),ann2,I began making fish curry for my boyfriend and I.,I decided not to read a recipe since I've made...,yes,['confident'],"{'joy': 2, 'trust': 3, 'fear': 0, 'surprise': ..."
...,...,...,...,...,...,...,...,...,...,...
42949,42949,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,4,Marcus,ann1,Marcus was collecting shells on the beach.|He ...,Suddenly he felt a sharp pinch.,yes,['sore'],"{'joy': 2, 'trust': 2, 'fear': 2, 'surprise': ..."
42950,42950,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,4,Marcus,ann2,Marcus was collecting shells on the beach.|He ...,Suddenly he felt a sharp pinch.,yes,['surprised'],"{'joy': 2, 'trust': 0, 'fear': 2, 'surprise': ..."
42951,42951,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,5,Marcus,ann0,Marcus was collecting shells on the beach.|He ...,A crab was inside the shell pinching his leg.,yes,['shocked'],"{'joy': 0, 'trust': 0, 'fear': 2, 'surprise': ..."
42952,42952,2aa1aca3-9264-4e27-9ebb-fa5de8e7e84e,5,Marcus,ann1,Marcus was collecting shells on the beach.|He ...,A crab was inside the shell pinching his leg.,yes,['surprised'],"{'joy': 0, 'trust': 0, 'fear': 2, 'surprise': ..."


In [68]:
import ast
trainDatasetProcessed = DataFrame({'sentence' : trainDatasetOriginal['sentence'], 
                                   'joy'  : [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['joy'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])], 
                                   'trust' : [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['trust'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])], 
                                   'fear'  : [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['fear'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])],
                                   'surprise': [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['surprise'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])],
                                   'sadness': [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['sadness'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])],
                                   'disgust': [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['disgust'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])],
                                   'anger': [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['anger'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])],
                                   'anticipation': [0 if (ast.literal_eval(trainDatasetOriginal['plutchik'][index])['anticipation'] == 0) else 1 for index in range(trainDatasetOriginal.shape[0])]})
testDatasetProcessed = DataFrame({'sentence' : testDatasetOriginal['sentence'], 
                                   'joy'  : [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['joy'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])], 
                                   'trust' : [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['trust'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])], 
                                   'fear'  : [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['fear'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])],
                                   'surprise': [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['surprise'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])],
                                   'sadness': [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['sadness'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])],
                                   'disgust': [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['disgust'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])],
                                   'anger': [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['anger'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])],
                                   'anticipation': [0 if (ast.literal_eval(testDatasetOriginal['plutchik'][index])['anticipation'] == 0) else 1 for index in range(testDatasetOriginal.shape[0])]})


In [69]:
trainDatasetProcessed.to_csv(f'./dataset/emoDetect-{fileTag}-train.csv')
trainDatasetProcessed

,sentence,joy,trust,fear,surprise,sadness,disgust,anger,anticipation
0,I began making fish curry for my boyfriend and I.,1,1,0,1,0,0,0,1
1,I began making fish curry for my boyfriend and I.,1,0,0,0,0,0,0,0
2,I began making fish curry for my boyfriend and I.,1,0,0,0,0,0,0,1
3,I decided not to read a recipe since I've made...,1,1,0,0,0,0,0,1
4,I decided not to read a recipe since I've made...,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
42949,Suddenly he felt a sharp pinch.,1,1,1,0,1,1,1,1
42950,Suddenly he felt a sharp pinch.,1,0,1,1,0,0,0,1
42951,A crab was inside the shell pinching his leg.,0,0,1,1,0,0,0,1
42952,A crab was inside the shell pinching his leg.,0,0,1,1,0,0,0,1


In [70]:
testDatasetProcessed.to_csv(f'./dataset/emoDetect-{fileTag}-test.csv')
testDatasetProcessed

,sentence,joy,trust,fear,surprise,sadness,disgust,anger,anticipation
0,A cook was carrying an armful of oranged in th...,1,1,0,1,0,0,0,1
1,A cook was carrying an armful of oranged in th...,1,0,0,0,0,0,0,1
2,He dropped one on the floor by accident.,0,0,1,1,0,0,0,1
3,He dropped one on the floor by accident.,0,0,0,0,0,1,1,0
4,He dropped one on the floor by accident.,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
43190,She had been dieting for weeks and was starving.,1,1,0,0,0,0,0,1
43191,"After she stuffed herself, she regretted it.",0,0,0,0,0,1,0,0
43192,"After she stuffed herself, she regretted it.",0,0,0,0,1,0,0,0
43193,Rosemary swore to eat healthier tomorrow.,0,0,0,0,1,1,1,0


# Start convert and train

In [59]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'train': f'./dataset/emoDetect-{fileTag}-train.csv', 
                                           'test': f'./dataset/emoDetect-{fileTag}-test.csv'})
dataset

Using custom data configuration default-3e6657a132cbe82c
Reusing dataset csv (C:\Users\JAM_0\.cache\huggingface\datasets\csv\default-3e6657a132cbe82c\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'sentence', 'joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation'],
        num_rows: 11610
    })
    test: Dataset({
        features: ['Unnamed: 0', 'sentence', 'joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust', 'anger', 'anticipation'],
        num_rows: 11129
    })
})

Let's check the first example of the training split:

In [60]:
example = dataset['train'][0]
example

{'Unnamed: 0': 0,
 'sentence': 'I began making fish curry for my boyfriend and I.',
 'joy': 1,
 'trust': 1,
 'fear': 0,
 'surprise': 1,
 'sadness': 0,
 'disgust': 0,
 'anger': 0,
 'anticipation': 1}

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [61]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Unnamed: 0', 'sentence']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['joy',
 'trust',
 'fear',
 'surprise',
 'sadness',
 'disgust',
 'anger',
 'anticipation']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [62]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_data(examples):
    # take a batch of texts
    text = examples["sentence"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding["labels"] = labels_matrix.tolist()
    return encoding

loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at C:\Users\JAM_0/.cache\huggingface\transformers\4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "ti

In [63]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Loading cached processed dataset at C:\Users\JAM_0\.cache\huggingface\datasets\csv\default-3e6657a132cbe82c\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-b8f84e6f4a47c3f8.arrow
Loading cached processed dataset at C:\Users\JAM_0\.cache\huggingface\datasets\csv\default-3e6657a132cbe82c\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-7d7653b6f5d9c429.arrow


In [64]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [65]:
tokenizer.decode(example['input_ids'])

'[CLS] i began making fish curry for my boyfriend and i. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [66]:
example['labels']

[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

In [67]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['joy', 'trust', 'surprise', 'anticipation']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [68]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

In [69]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id, 
                                                           ignore_mismatched_sizes=True)

loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at C:\Users\JAM_0/.cache\huggingface\transformers\4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "joy",
    "1": "trust",
    "2": "fear",
    "3": "surprise",
    "4": "sadness",
    "5": "disgust",
    "6": "anger",
    "7": "anticipation"
  },
  "initializer_range": 0.02,
  "label2id": {
    "anger": 6,
    "anticipation": 7,
    "disgust": 5,
    "fear": 2,
    "joy": 0,
    "sadness": 4,
    "surprise": 3,
    "trust": 1
  },
  "max_position_e

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [70]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=9e-7,
    learning_rate=6e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [71]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [72]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [73]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1045,  2211,  2437,  3869, 15478,  2005,  2026,  6898,  1998,
         1045,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [74]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6971, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.3067, -0.0404,  0.0743, -0.0565,  0.2295, -0.1519,  0.2057,  0.1446]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [75]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [76]:
trainer.train()

C:\Python\miniconda3\envs\pytorchEnvWithDataSci\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11610
  Num Epochs = 16
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11616


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.568500,0.513809,0.687942,0.732518,0.149340
2,0.517700,0.498210,0.702404,0.745091,0.169647
3,0.474700,0.493014,0.707881,0.750143,0.178453
4,0.461100,0.495391,0.708241,0.749431,0.179621
5,0.435600,0.497070,0.708379,0.751167,0.184114
6,0.423000,0.504051,0.708572,0.750299,0.182676
7,0.407400,0.510026,0.707999,0.750787,0.183664
8,0.396300,0.512795,0.707875,0.750188,0.183754
9,0.386700,0.516743,0.703349,0.747136,0.181777
10,0.378300,0.524007,0.705579,0.748251,0.180340


***** Running Evaluation *****
  Num examples = 11129
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english\checkpoint-726
Configuration saved in bert-finetuned-sem_eval-english\checkpoint-726\config.json
Model weights saved in bert-finetuned-sem_eval-english\checkpoint-726\pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english\checkpoint-726\tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english\checkpoint-726\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 11129
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english\checkpoint-1452
Configuration saved in bert-finetuned-sem_eval-english\checkpoint-1452\config.json
Model weights saved in bert-finetuned-sem_eval-english\checkpoint-1452\pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english\checkpoint-1452\tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english\che

TrainOutput(global_step=11616, training_loss=0.41031546599280405, metrics={'train_runtime': 1103.9228, 'train_samples_per_second': 168.273, 'train_steps_per_second': 10.522, 'total_flos': 6152444243804160.0, 'train_loss': 0.41031546599280405, 'epoch': 16.0})

In [77]:
import pandas as pd
dataLog = pd.DataFrame(trainer.state.log_history)
dataLog.to_csv(f'./trainingMetric/TI-{model_checkpoint}-{fileTag}.csv')

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 11])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['joy', 'optimism']
